In [1]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 110.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97

In [2]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 31.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.2
    Uninstalling protobuf-3.20.2:
      Successfully uninstalled protobuf-3.20.2


In [3]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-CoLA")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-CoLA")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Construct our four components for `Attack`

from textattack.constraints.pre_transformation import RepeatModification, StopwordModification

from textattack.constraints.semantics import WordEmbeddingDistance

In [6]:
goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

constraints = [

    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.7)

]

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:13<00:00, 36.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmprwrqbgft.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [7]:
transformation = textattack.transformations.word_swaps.word_swap_homoglyph_swap.WordSwapHomoglyphSwap()
search_method = textattack.search_methods.BeamSearch(beam_width=8)

In [8]:
#attack = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(model_wrapper)
attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

#datasets.load_dataset('emotion', split =['test'])
#dataset=datasets.load_dataset('sms_spam', split =['test'])
dataset = textattack.datasets.HuggingFaceDataset("linxinyuan/cola", dataset_columns=(["text"],'label'))
#attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cola downloaded and prepared to /root/.cache/huggingface/datasets/linxinyuan___cola/default/0.0.0/0871d55203d4de46ef1815400998ed8f219236694f0d03786bde849741f04cd4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset linxinyuan/cola, split train.


In [11]:

attack_args = textattack.AttackArgs(

    num_examples=100,

    log_to_csv="advsmish.csv",

    checkpoint_interval=2,

    checkpoint_dir="checkpoints",

    disable_stdout=True

)

attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset() 

textattack: Logging to CSV at path advsmish.csv


Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHomoglyphSwap
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   2%|▏         | 2/100 [00:02<01:38,  1.01s/it]textattack: Saving checkpoint under "checkpoints/1686027010767.ta.chkpt" at 2023-06-06 04:50:10 after 2 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:   4%|▍         | 4/100 [00:03<01:22,  1.17it/s]textattack: Saving checkpoint under "checkpoints/1686027012186.ta.chkpt" at 2023-06-06 04:50:12 after 4 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 0 / 0 / 6:   6%|▌         | 6/100 [00:04<01:09,  1.35it/s]textattack: Saving checkpoint under "checkpoints/1686027013187.ta.chkpt" at 2023-06-06 04:50:13 after 6 attacks.


...

[Succeeded / Failed / Skipped / Total] 88 / 2 / 4 / 94:  94%|█████████▍| 94/100 [01:25<00:05,  1.10it/s]textattack: Saving checkpoint under "checkpoints/1686027094454.ta.chkpt" at 2023-06-06 04:51:34 after 94 attacks.


[Succeeded / Failed / Skipped / Total] 90 / 2 / 4 / 96:  96%|█████████▌| 96/100 [01:27<00:03,  1.10it/s]textattack: Saving checkpoint under "checkpoints/1686027095806.ta.chkpt" at 2023-06-06 04:51:35 after 96 attacks.


[Succeeded / Failed / Skipped / Total] 92 / 2 / 4 / 98:  98%|█████████▊| 98/100 [01:28<00:01,  1.11it/s]textattack: Saving checkpoint under "checkpoints/1686027096937.ta.chkpt" at 2023-06-06 04:51:36 after 98 attacks.


[Succeeded / Failed / Skipped / Total] 94 / 2 / 4 / 100: 100%|██████████| 100/100 [01:29<00:00,  1.12it/s]textattack: Saving checkpoint under "checkpoints/1686027098337.ta.chkpt" at 2023-06-06 04:51:38 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 94 / 2 / 4 / 100: 100%|██████████| 100/100 [01:29<00:00,  1.12it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 94     |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 96.0%  |
| Accuracy under attack:        | 2.0%   |
| Attack success rate:          | 97.92% |
| Average perturbed word %:     | 18.74% |
| Average num. words per input: | 6.43   |
| Avg num queries:              | 33.79  |
+-------------------------------+--------+


In [10]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attack_results=attacker.attack_dataset()

Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHomoglyphSwap
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:01<00:10,  1.13s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (95%)]] --> [[0 (82%)]]

Our friends won't buy this analysis, let [[alone]] the next one we propose.

Our friends won't buy this analysis, let [[aⅼone]] the next one we propose.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:01<00:07,  1.04it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (97%)]] --> [[0 (69%)]]

[[One]] more pseudo generalization and I'm giving up.

[[Onе]] more pseudo generalization and I'm giving up.




...

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (90%)]] --> [[0 (77%)]]

Bill coughed his [[way]] out of the restaurant.

Bill coughed his [[ԝay]] out of the restaurant.




[Succeeded / Failed / Skipped / Total] 9 / 0 / 0 / 9:  90%|█████████ | 9/10 [00:06<00:00,  1.49it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (97%)]] --> [[0 (90%)]]

We're dancing the night [[away]].

We're dancing the night [[ɑway]].




[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10: 100%|██████████| 10/10 [00:06<00:00,  1.49it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[1 (96%)]] --> [[0 (76%)]]

[[Herman]] hammered the metal flat.

[[Heⲅman]] hammered the metal flat.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 14.88% |
| Average num. words per input: | 7.4    |
| Avg num queries:              | 25.4   |
+-------------------------------+--------+
